In [1]:
from selenium import webdriver
import pandas as pd

In [2]:
# ランキング表のリンク一覧（日付順で並んでいる表）をリストにして返す
# リスト構造: [日付, URL(XHTML版ランキング), URL(テキスト版ランキング)]
def extract_ranking_links_as_list():
    browser = webdriver.Chrome()
    browser.get('https://www.aozora.gr.jp/access_ranking/')
    
    
    tables = browser.find_elements_by_tag_name('table')

    # 2番目のtableに最新のランキング項目が入っている
    table_index = 1
    table = tables[table_index]

    # ランキング表のリンク一覧を抽出する
    trs = table.find_elements_by_tag_name('tr')

    rank_list = []
    for tr in trs:    
        tds = tr.find_elements_by_tag_name('td')
        str_date = tds[0].text
        str_url_xhtml = tds[1].find_element_by_tag_name('a').get_attribute('href')
        str_url_txt = tds[2].find_element_by_tag_name('a').get_attribute('href')

        rank_list.append([str_date, str_url_xhtml, str_url_txt])
    
    browser.quit()
    return rank_list

In [3]:
link_list = extract_ranking_links_as_list()
link_list

[['2021年8月',
  'https://www.aozora.gr.jp/access_ranking/2021_08_xhtml.html',
  'https://www.aozora.gr.jp/access_ranking/2021_08_txt.html'],
 ['2021年7月',
  'https://www.aozora.gr.jp/access_ranking/2021_07_xhtml.html',
  'https://www.aozora.gr.jp/access_ranking/2021_07_txt.html'],
 ['2021年6月',
  'https://www.aozora.gr.jp/access_ranking/2021_06_xhtml.html',
  'https://www.aozora.gr.jp/access_ranking/2021_06_txt.html'],
 ['2021年5月',
  'https://www.aozora.gr.jp/access_ranking/2021_05_xhtml.html',
  'https://www.aozora.gr.jp/access_ranking/2021_05_txt.html'],
 ['2021年4月',
  'https://www.aozora.gr.jp/access_ranking/2021_04_xhtml.html',
  'https://www.aozora.gr.jp/access_ranking/2021_04_txt.html'],
 ['2021年3月',
  'https://www.aozora.gr.jp/access_ranking/2021_03_xhtml.html',
  'https://www.aozora.gr.jp/access_ranking/2021_03_txt.html'],
 ['2021年2月',
  'https://www.aozora.gr.jp/access_ranking/2021_02_xhtml.html',
  'https://www.aozora.gr.jp/access_ranking/2021_02_txt.html'],
 ['2021年1月',
  'http

In [4]:
KEY_TIMESTAMP = 'タイムスタンプ'
KEY_URL_XHTML = 'URL (XHTML版)'
KEY_URL_TXT = 'URL (テキスト版)'
KEY_DATE = '年月'

# ランキング表のリンク一覧をDataFrameとして返す
def extract_ranking_links_as_data_frame():
    link_list = extract_ranking_links_as_list()

    df = pd.DataFrame(link_list, columns=[KEY_DATE, KEY_URL_XHTML, KEY_URL_TXT])
    df[KEY_TIMESTAMP] = pd.to_datetime(df[KEY_DATE], format='%Y年%m月')

    # 新しい方を上にする
    df_sorted = df.sort_values(by=KEY_TIMESTAMP, ascending=False) 
    return df_sorted

In [5]:
df = extract_ranking_links_as_data_frame()
df

,年月,URL (XHTML版),URL (テキスト版),タイムスタンプ
0,2021年8月,https://www.aozora.gr.jp/access_ranking/2021_0...,https://www.aozora.gr.jp/access_ranking/2021_0...,2021-08-01
1,2021年7月,https://www.aozora.gr.jp/access_ranking/2021_0...,https://www.aozora.gr.jp/access_ranking/2021_0...,2021-07-01
2,2021年6月,https://www.aozora.gr.jp/access_ranking/2021_0...,https://www.aozora.gr.jp/access_ranking/2021_0...,2021-06-01
3,2021年5月,https://www.aozora.gr.jp/access_ranking/2021_0...,https://www.aozora.gr.jp/access_ranking/2021_0...,2021-05-01
4,2021年4月,https://www.aozora.gr.jp/access_ranking/2021_0...,https://www.aozora.gr.jp/access_ranking/2021_0...,2021-04-01
5,2021年3月,https://www.aozora.gr.jp/access_ranking/2021_0...,https://www.aozora.gr.jp/access_ranking/2021_0...,2021-03-01
6,2021年2月,https://www.aozora.gr.jp/access_ranking/2021_0...,https://www.aozora.gr.jp/access_ranking/2021_0...,2021-02-01
7,2021年1月,https://www.aozora.gr.jp/access_ranking/2021_0...,https://www.aozora.gr.jp/access_ranking/2021_0...,2021-01-01


In [6]:
# DataFrameから特定のタイムスタンプを持つ行を抽出する
def extract_df_by_timestamp(df, timestamp):
    return df[df[KEY_TIMESTAMP] == timestamp]

In [7]:
df_target = extract_df_by_timestamp(df, '2021-06-01')
df_target

,年月,URL (XHTML版),URL (テキスト版),タイムスタンプ
2,2021年6月,https://www.aozora.gr.jp/access_ranking/2021_0...,https://www.aozora.gr.jp/access_ranking/2021_0...,2021-06-01


In [8]:
# 特定のタイムスタンプにおいて、URL（XHTML版、テキスト版のそれぞれ）を返す
def get_urls_by_timestamp(df, timestamp):
    df_target = extract_df_by_timestamp(df, timestamp).iloc[0]
    return df_target[KEY_URL_XHTML], df_target[KEY_URL_TXT]

In [9]:
url_rank_xhtml, url_rank_txt = get_urls_by_timestamp(df, '2021-06-01')
url_rank_xhtml

'https://www.aozora.gr.jp/access_ranking/2021_06_xhtml.html'

In [10]:
# ランキング表のデータフレームを取得する
def get_df_ranking(url):
    browser = webdriver.Chrome()
    browser.get(url)

    table = browser.find_elements_by_tag_name("table")[0]
    table_html = table.get_attribute('outerHTML')
    browser.quit()

    table_pd = pd.read_html(table_html, header=0, index_col=0)

    return table_pd[0]

In [11]:
# ランキング表（DataFrame)
df_ranking = get_df_ranking(url_rank_xhtml)
df_ranking

,作品名副題,著者名,アクセス数
ランキング,,,
1,山月記,中島 敦,36942
2,〔雨ニモマケズ〕,宮沢 賢治,30612
3,羅生門,芥川 竜之介,30182
4,走れメロス,太宰 治,20112
5,風の又三郎,宮沢 賢治,15846
...,...,...,...
496,「人間宣言」のうちそと,前田 多門,386
497,城のある町にて,梶井 基次郎,383
498,俳諧大要,正岡 子規,383


In [16]:
# 作品名のリスト
list_titles = df_ranking['作品名副題'].tolist()
list_titles

['山月記',
 '〔雨ニモマケズ〕',
 '羅生門',
 '走れメロス',
 '風の又三郎',
 'ドグラ・マグラ',
 '吾輩は猫である',
 '銀河鉄道の夜',
 '檸檬',
 '夢十夜',
 'こころ',
 '蜘蛛の糸',
 '注文の多い料理店',
 '草枕',
 '方丈記',
 '人間失格',
 '坊っちゃん',
 'やまなし',
 '舞姫',
 'よだかの星',
 '食物として',
 'あのときの王子くん',
 '駈込み訴え',
 '『春と修羅』',
 '学問のすすめ',
 '鼻',
 '土佐日記',
 '桜の樹の下には',
 '斜陽',
 '高瀬舟',
 '地獄変',
 '瓶詰地獄',
 '智恵子抄',
 '河童',
 '手袋を買いに',
 'オツベルと象',
 '桃太郎',
 '女生徒',
 '月に吠える02 月に吠える',
 '星めぐりの歌',
 '遠野物語',
 '陰翳礼讃',
 '一握の砂',
 '芋粥',
 'トロッコ',
 '銀河鉄道の夜',
 '蟹工船',
 '夏の葬列',
 '蜜柑',
 'セロ弾きのゴーシュ',
 'ロミオとヂュリエット03 ロミオとヂュリエット',
 'ごん狐',
 '人間椅子',
 '春琴抄',
 '三四郎',
 '刺青',
 '金色夜叉',
 '杜子春',
 'セメント樽の中の手紙',
 '毒もみのすきな署長さん',
 '葉',
 '桜桃',
 '桃太郎',
 '痴人の愛',
 '名人伝',
 '山羊の歌',
 'たけくらべ',
 '蒲団',
 '恩讐の彼方に',
 '形',
 '富嶽百景',
 '赤い蝋燭と人魚',
 '藪の中',
 '変身',
 '尾崎放哉選句集',
 '高瀬舟',
 '呪咀',
 '夜明け前01 第一部上',
 'トカトントン',
 'アリスはふしぎの国で',
 '藪の中',
 '武蔵野',
 '少女地獄',
 '堕落論',
 '或阿呆の一生',
 '桜の森の満開の下',
 '歯車',
 '故郷',
 'あたらしい憲法のはなし',
 '舞姫',
 '黒死館殺人事件',
 '押絵と旅する男',
 'ポラーノの広場',
 '私の個人主義',
 'どんぐりと山猫',
 'グスコーブドリの伝記',
 '文字禍',
 '外科室',
 'ア、秋',
 '破戒',
 '戦争責任者の問題',
 '二銭銅貨'

In [13]:
df_groupby_author = df_ranking.groupby('著者名').sum().sort_values('アクセス数', ascending=False)
df_groupby_author

,アクセス数
著者名,
宮沢 賢治,119958
芥川 竜之介,102078
太宰 治,71052
夏目 漱石,68276
中島 敦,45252
...,...
山崎 富栄,393
永井 隆,393
岸田 国士,393


In [36]:
# 著者名のリスト
list_authors = df_groupby_author.reset_index()['著者名'].tolist()
list_authors

['宮沢 賢治',
 '芥川 竜之介',
 '太宰 治',
 '夏目 漱石',
 '中島 敦',
 '夢野 久作',
 '森 鴎外',
 '江戸川 乱歩',
 '梶井 基次郎',
 '谷崎 潤一郎',
 '新美 南吉',
 '泉 鏡花',
 '鴨 長明',
 '坂口 安吾',
 '樋口 一葉',
 '楠山 正雄',
 'サン＝テグジュペリ アントワーヌ・ド',
 '萩原 朔太郎',
 '福沢 諭吉',
 '小川 未明',
 '菊池 寛',
 '柳田 国男',
 '島崎 藤村',
 '高村 光太郎',
 'アンデルセン ハンス・クリスチャン',
 '紀 貫之',
 '魯迅',
 '有島 武郎',
 '田山 花袋',
 'グリム ヤーコプ・ルートヴィッヒ・カール\u3000グリム ヴィルヘルム・カール',
 '山川 方夫',
 '石川 啄木',
 '国木田 独歩',
 '中原 中也',
 'ポー エドガー・アラン',
 '小林 多喜二',
 '横光 利一',
 'シェークスピア ウィリアム',
 '山本 周五郎',
 '北原 白秋',
 'ラヴクラフト ハワード・フィリップス',
 '尾崎 紅葉',
 '葉山 嘉樹',
 '寺田 寅彦',
 '幸田 露伴',
 '岡本 かの子',
 '与謝野 晶子',
 '北大路 魯山人',
 '正岡 子規',
 'カフカ フランツ',
 '尾崎 放哉',
 '小泉 八雲',
 '土谷 麓',
 'キャロル ルイス',
 '堀 辰雄',
 '吉川 英治',
 '文部省',
 '小栗 虫太郎',
 '室生 犀星',
 '太 安万侶\u3000稗田 阿礼',
 'スティーブンソン ロバート・ルイス',
 '山村 暮鳥',
 '伊丹 万作',
 'ドストエフスキー フィヨードル・ミハイロヴィチ',
 'ドイル アーサー・コナン',
 '二葉亭 四迷',
 '八木 重吉',
 '三遊亭 円朝',
 'テニソン アルフレッド',
 'チェーホフ アントン',
 '織田 作之助',
 '岡本 綺堂',
 'ヘミングウェイ アーネスト・ミラー',
 '泉 鏡花\u3000泉 鏡太郎',
 '巌谷 小波',
 'ポター ビアトリクス',
 '三好 達治',
 'ゲーテ ヨハン・ヴォルフガング・フォン',
 '南部 修太郎',
 '永井 荷風',
 'ツ

In [ ]:
# TODO: 書籍名とアクセス数でグラフを作ってみる